In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import dtale
from autoviz.AutoViz_Class import AutoViz_Class
import sweetviz as sv
%matplotlib inline


In [ ]:
df = pd.read_csv('Pokemon.csv')
#autoviz = AutoViz_Class().AutoViz('Pokemon.csv')
sweet_report = sv.analyze(df)
sweet_report.show_notebook()

In [ ]:
df['Legendary'] = df['Legendary'].astype(int)
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
df.plot(kind='scatter', x = 'HP',y = 'Attack',s = 100,
c = 'Legendary',cmap='winter',alpha = 0.5,ax = ax)

In [ ]:
df.loc[:,'HP':'Speed'].corr()

In [ ]:
sns.pairplot(df.loc[:,'HP':'Speed'])

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
x_pca = pca.fit_transform(df.loc[:,'HP':'Speed'])
df_pca = pd.DataFrame(x_pca,columns=['1st','2nd','3rd','4th','5th','6th'])
df_pca.head()

In [ ]:
pca.explained_variance_ratio_#寄与率
#上位３つの変数で寄与率８割を占めているということは図示できる

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.hstack([0,pca.explained_variance_ratio_.cumsum()]))#0から
plt.xlabel('n_components',fontsize=15)
plt.ylabel('explained_variance_ratio_',fontsize=15)
plt.show

In [ ]:
pca.explained_variance_ratio_.cumsum()#累積和

In [ ]:
pca.components_#主成分負荷量　主成分得点と観測変数との相関係数のこと

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(pca.components_,
cmap='Blues',
annot=True,annot_kws={'size':20},
fmt = '1.1f',
xticklabels=df.loc[:,'HP':'Speed'].columns,
yticklabels=['1st','2nd','3rd','4th','5th','6th'])

In [ ]:
df_pca['Legendary'] = df['Legendary']

fig,ax = plt.subplots(figsize=(12,8))
df_pca.plot(kind='scatter',
x='1st',
y='2nd',
s=100,
c='Legendary',
cmap='winter',
alpha=0.5,
ax=ax)

In [ ]:
#特徴量抽出
df1 = pd.read_csv('train.csv')
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.isnull().sum()[df1.isnull().sum()>0]

In [ ]:
print('before:', df.shape)
df1 = df1.drop(columns=['LotFrontage','Alley','FireplaceQu','PoolQC','Fence','MiscFeature'])
print('after:',df1.shape)

In [ ]:
df1.dtypes.value_counts()

In [ ]:
all_columns = df1.columns.values
non_cat_cols = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
                "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
                "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                "ScreenPorch","PoolArea", "MiscVal"]

cat_cols = [col for col in all_columns if col not in non_cat_cols]


In [ ]:
nan_cols = df1.isnull().sum()[df1.isnull().sum() > 0].index
for col in nan_cols:
    if col in non_cat_cols:
        df1[col] = df1[col].fillna(df1[col].median())
    else:
        df1[col] = df1[col].fillna(df1[col].mode()[0])
df1 = pd.get_dummies(df1)
df1

In [ ]:
X = df1.drop(columns=['Id','SalePrice'])
y = df1['SalePrice']

x = np.log1p(X)
y = np.log1p(y)
X[X == -np.inf] = 0
x.shape


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=0)

In [ ]:
pca = PCA()

In [ ]:
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.fit_transform(X_test)

pca.explained_variance_ratio_.cumsum()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.hstack([0,pca.explained_variance_ratio_.cumsum()]))#0から
plt.xlabel('n_components',fontsize=15)
plt.ylabel('explained_variance_ratio_',fontsize=15)
plt.show

In [ ]:
X_train_pca = X_train_pca[:,:8]
X_test_pca = X_test_pca[:,:8]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_absolute_error

In [ ]:
%%time

cv = KFold(n_splits=5,shuffle=True,random_state=0)
rf_score = np.sqrt(-cross_val_score(RandomForestRegressor(n_estimators=500,max_depth=6),
                                                X = X_train,
                                                y = y_train,
                                                cv = cv,
                                                scoring = 'neg_mean_squared_error'))
print(rf_score.mean())

In [ ]:
%%time

cv = KFold(n_splits=5,shuffle=True,random_state=0)
rf_score_pca = np.sqrt(-cross_val_score(RandomForestRegressor(n_estimators=500,max_depth=6),
                                                X = X_train_pca,
                                                y = y_train,
                                                cv = cv,
                                                scoring = 'neg_mean_squared_error'))

print(rf_score_pca.mean())

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

rf_params_grid = {
    'max_depth' : range(5,11)
}

rf_score = np.sqrt(-cross_val_score(GridSearchCV(RandomForestRegressor(n_estimators=500,max_depth=6),
                                                param_grid=rf_params_grid,
                                                cv=cv),
                                                X = X_train_pca,
                                                y = y_train,
                                                cv = cv,
                                                scoring = 'neg_mean_squared_error'))



